**Source:** [Estimating a probability distribution non-parametrically with a kernel density estimation](https://ipython-books.github.io/76-estimating-a-probability-distribution-nonparametrically-with-a-kernel-density-estimation/) ( Uses `scipy.stats.gaussian_kde`)

In [ ]:
import os

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as st
from matplotlib.colors import ListedColormap
from sklearn.neighbors import KernelDensity

**Original article uses a weather dataset: it contains information about most storms since 1848. A single storm may appear multiple times across several consecutive days.**

In [ ]:
# # www.ncdc.noaa.gov/ibtracs/index.php?name=wmo-data
# url = "https://raw.githubusercontent.com/ipython-books/cookbook-2nd-data/master/Allstorms.ibtracs_wmo.v03r05.csv"
# df = pd.read_csv(url)
# df[df.columns[[0, 1, 3, 8, 9]]].head()

In [ ]:
# We use pandas groupby() function to obtain the average location of every storm
# dfs = df.groupby('Serial_Num')
# pos = dfs[['Latitude', 'Longitude']].mean()
# x = pos.Longitude.values
# y = pos.Latitude.values
# pos.head()

## Coordinates of samples from Pangaea
We will adapt the code for plotting our image sample locations

In [ ]:
# Load files
DATA_DIR = "../query-outputs/"
files = os.listdir(DATA_DIR)
df_list = [pd.read_csv(os.path.join(DATA_DIR, f)) for f in files]
print(f"Total {len(df_list)} files loaded.")

# Check if all files have lat, lon data
def has_lat_lon(frame):
    cond1 = any([col == "Latitude" for col in frame.columns])
    cond2 = any([col == "Longitude" for col in frame.columns])
    return cond1 and cond2


print("All files have lat/lon columns:", all([has_lat_lon(df) for df in df_list]))
# Join all datasets
all_dfs = pd.concat(df_list)

# Values for plotting
x = all_dfs["Longitude"].dropna().to_numpy()
y = all_dfs["Latitude"].dropna().to_numpy()
print("x.shape:", x.shape, "y.shape:", y.shape)

## 1. Scatter plot

In [ ]:
# Transform
geo = ccrs.Geodetic()
# Projection
crs = ccrs.EqualEarth()

# We create the map plot.
fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=crs)

# We display the world map picture.
ax.stock_img()
# We display the storm locations.
ax.scatter(x, y, color="r", alpha=0.15, transform=geo)
ax.set_title("Spatial distribution of image samples")
plt.show()

## 2. KDE SciPy

**Transform the image positions from the geodetic coordinate system (longitude and latitude) into the map's coordinate system**

In [ ]:
h = crs.transform_points(geo, x, y)[:, :2].T
h.shape

**Now, we perform the kernel density estimation on our (2, N) array.** using `scipy.stats.gaussian_kde`

In [ ]:
kde = st.gaussian_kde(h)

**The `gaussian_kde()` routine returned a Python function. To see the results on a map, we need to evaluate this function on a 2D grid spanning the entire map. We create this grid with `meshgrid()`, and we pass the x and y values to the `kde()` function:**

In [ ]:
k = 100
# Coordinates of the four corners of the map.
x0, x1, y0, y1 = ax.get_extent()
# We create the grid.
tx, ty = np.meshgrid(np.linspace(x0, x1, 2 * k), np.linspace(y0, y1, k))
# We reshape the grid for the kde() function.
mesh = np.vstack((tx.ravel(), ty.ravel()))
# We evaluate the kde() function on the grid.
v = kde(mesh).reshape((k, 2 * k))

**Before displaying the KDE heatmap on the map, we need to use a special colormap with a transparent channel. This will allow us to superimpose the heatmap on the stock image:**

In [ ]:
# https://stackoverflow.com/a/37334212/1595060
cmap = plt.get_cmap("Reds")
my_cmap = cmap(np.arange(cmap.N))
my_cmap[:, -1] = np.linspace(0, 1, cmap.N)
my_cmap = ListedColormap(my_cmap)

**Finally, we display the estimated density with `imshow()` or `contourf()`:**

In [ ]:
fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=crs)
ax.stock_img()

# ax.imshow(v, origin='lower', cmap=my_cmap, extent=[x0, x1, y0, y1], interpolation='bilinear')
ax.contourf(
    tx,
    ty,
    v,
    cmap=my_cmap,
    extent=[x0, x1, y0, y1],
    levels=np.linspace(0, v.max(), 25),
    origin="lower",
)
# ax.scatter(x, y, color='r', s=5, alpha=0.15, transform=geo)
ax.set_title("KDE of image sample spatial distribution")
plt.show()

### Side by side

In [ ]:
fig = plt.figure(figsize=(25, 8))

# KDE plot
ax = fig.add_subplot(121, projection=crs)
ax.stock_img()
ax.set_title("KDE plot", fontsize=20)
ax.contourf(
    tx,
    ty,
    v,
    cmap=my_cmap,
    levels=np.linspace(0, v.max(), 25),
    origin="lower",
    extent=[x0, x1, y0, y1],
    interpolation="bilinear",
)
# Scatter plot
ax = fig.add_subplot(122, projection=crs)
ax.stock_img()
ax.set_title("Scatter plot", fontsize=20)
ax.scatter(x, y, color="r", alpha=0.25, transform=geo)

## 3. KDE Sklearn
The function below is based on the `kde2()` function from the following article: [Two-dimensional kernel density estimate: comparing scikit-learn and scipy](https://gist.github.com/daleroberts/7a13afed55f3e2388865b0ec94cd80d2)

In [ ]:
def kde_sklearn(x, y, metric="euclidean"):
    xy = np.vstack([x, y]).T
    # Bandwidth calculation
    d = xy.shape[0]
    n = xy.shape[1]
    bw = (n * (d + 2) / 4.0) ** (-1.0 / (d + 4))  # silverman
    # bw = n**(-1./(d+4)) # scott
    print(f"bw: {bw}, metric: {metric}")
    # KDE
    kde = KernelDensity(
        bandwidth=bw, metric=metric, kernel="gaussian", algorithm="ball_tree"
    )
    kde.fit(xy)
    # Extent
    xmin = x.min()
    xmax = x.max()
    ymin = y.min()
    ymax = y.max()
    # Mesh grid
    X, Y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
    positions = np.vstack([X.ravel(), Y.ravel()])
    # Z heights
    Z = np.reshape(np.exp(kde.score_samples(positions.T)), X.shape)
    return X, Y, Z

In [ ]:
X, Y, Z = kde_sklearn(x, y, metric="euclidean")
X2, Y2, Z2 = kde_sklearn(x, y, metric="haversine")

**euclidean**

In [ ]:
fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=crs)
ax.stock_img()
ax.imshow(
    Z, cmap=my_cmap, extent=[x0, x1, y0, y1], interpolation="bilinear", origin="lower"
)
# ax.contourf(X, Y, Z, cmap=my_cmap, extent=[x0, x1, y0, y1],
#             levels=np.linspace(0, v.max(), 25),
#             interpolation='bilinear',
#             origin='lower')
# ax.scatter(x, y, color='r', alpha=0.25, transform=geo)
ax.set_title("KDE of image sample spatial distribution")
plt.show()

**haversine**

In [ ]:
fig = plt.figure(figsize=(25, 8))
ax = fig.add_subplot(projection=crs)
ax.stock_img()
ax.imshow(
    Z2, cmap=my_cmap, extent=[x0, x1, y0, y1], interpolation="bilinear", origin="lower"
)
# ax.contourf(X2, Y2, Z2, cmap=my_cmap, extent=[x0, x1, y0, y1],
#             levels=np.linspace(0, v.max(), 25),
#             interpolation='bilinear',
#             origin='lower')
# ax.scatter(x, y, color='r', alpha=0.25, transform=geo)
ax.set_title("KDE of image sample spatial distribution")
plt.show()